# Extract. Transform. Analysis.

In [1]:
import pandas as pd
import requests
import country_converter as coco
from bs4 import BeautifulSoup
import folium

### Extract

За основу беру таблицу Numbeo "Current Cost of Living Index", в котором на данный момент 506 городов и, пока, не понятно сколько стран и по сколько городов в этих странах.

In [2]:
url = 'https://www.numbeo.com/cost-of-living/rankings_current.jsp'
df = pd.read_html(url)[1]
df.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42
1,NaN,"Zurich, Switzerland",121.25,61.89,93.17,122.95,115.77,115.64
2,NaN,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13
3,NaN,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38
4,NaN,"Bern, Switzerland",113.85,36.83,77.41,115.93,106.74,116.09


### Transform

In [3]:
df.drop('Rank', axis=1, inplace=True)

Добавляю в таблицу ссылки на страницы данных городов со строгим указанием валюты USD

In [4]:
response = requests.get(url).text
soup = BeautifulSoup(response, 'lxml')
links = soup.find_all('td', class_='cityOrCountryInIndicesTable')

href_list = []
for i in links:
    href_list.append([i.find('a').text, i.find('a').get('href') + '?displayCurrency=USD'])
    
city_link = pd.DataFrame(href_list, columns=['City', 'link'])

df = df.merge(city_link, on='City', how='left')

### Разделить Город - Штат - Страна

In [5]:
split_table1 = df['City'].str.rsplit(', ', n=1, expand=True)
split_table1.columns=['city_state', 'country']

split_table2 = split_table1['city_state'].str.split(', ', n=1, expand=True)
split_table2.columns=['city', 'state']

df = pd.concat([split_table2, split_table1['country'], df],axis=1)

df['city'] = df.apply(lambda row: row.city.split(' (')[0], axis = 1) #убираю дубли городов в скобках

In [6]:
df.head()

,city,state,country,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,link
0,Hamilton,None,Bermuda,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,https://www.numbeo.com/cost-of-living/in/Hamil...
1,Zurich,None,Switzerland,"Zurich, Switzerland",121.25,61.89,93.17,122.95,115.77,115.64,https://www.numbeo.com/cost-of-living/in/Zuric...
2,Basel,None,Switzerland,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,https://www.numbeo.com/cost-of-living/in/Basel...
3,Zug,None,Switzerland,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38,https://www.numbeo.com/cost-of-living/in/Zug?d...
4,Bern,None,Switzerland,"Bern, Switzerland",113.85,36.83,77.41,115.93,106.74,116.09,https://www.numbeo.com/cost-of-living/in/Bern?...


### Добавляю alpha_3 код к странам
Добавляю через lambda к каждой строке. Не самый эффективный вариант, обрабатывается капждая строка, страны дублируются, но просто попробовать (merge уже был).

In [7]:
df['country_code'] = df.apply(lambda row: coco.convert(names=row.country, to='ISO3') , axis = 1)
df.head(3)

,city,state,country,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,link,country_code
0,Hamilton,None,Bermuda,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,https://www.numbeo.com/cost-of-living/in/Hamil...,BMU
1,Zurich,None,Switzerland,"Zurich, Switzerland",121.25,61.89,93.17,122.95,115.77,115.64,https://www.numbeo.com/cost-of-living/in/Zuric...,CHE
2,Basel,None,Switzerland,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,https://www.numbeo.com/cost-of-living/in/Basel...,CHE


### Добавляю код к штатам USA
Есть три значения штатов, не относящихся к штатам США.
Всем этим значениям присвою знанчение None.
А к штатам США добавлю "US- " - приведу к ISO стандарту обозначения штатов США.

In [8]:
df[df['state'].notnull() & (df['country'] != "United States")]

,city,state,country,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,link,country_code
54,Nanaimo,BC,Canada,"Nanaimo, BC, Canada",77.47,37.20,58.42,81.15,66.41,90.51,https://www.numbeo.com/cost-of-living/in/Nanai...,CAN
73,St. John's,Newfoundland and Labrador,Canada,"St. John's, Newfoundland and Labrador, Canada",75.16,22.95,50.46,74.43,71.00,141.04,https://www.numbeo.com/cost-of-living/in/St-Jo...,CAN
452,Batumi,Ajara,Georgia,"Batumi, Ajara, Georgia",32.25,12.79,23.04,31.50,27.24,26.18,https://www.numbeo.com/cost-of-living/in/Batum...,GEO


In [9]:
indexes = df[df['state'].notnull() & (df['country'] != "United States")]['state'].index
df.loc[indexes, 'state'] = None

df['state'] = df.apply(lambda row: f"US-{row['state']}" if row['state'] != None else None, axis = 1)

# Карта
В индексе Numbeo 505 городов, но не понятно, какие страны, много их или мало. Сколько городов в этих странах и в каких пропорциях. Суммарные цифры мне мало что скажут, гораздо понятнее будет отобразить их на карте.

In [10]:
map_df = df.groupby('country_code')['city'].nunique().reset_index().sort_values('city', ascending=False)

m = folium.Map(location=[37.87820990704326, 6.555063556986549], zoom_start=1.5)
folium.Choropleth(
    geo_data='data/world.geojson',
    name="choropleth",
    data=map_df,
    columns=['country_code', 'city'],
    key_on="feature.properties.ISO_A3", 
    fill_color="YlGn",
    nan_fill_color='pink',
    fill_opacity=0.8,
    bins=7,
    reset=True,
    highlight=True,
    legend_name='Count of cities'
).add_to(m)

![title](data/map.png)

На github интерактивная карта не отображается, поэтому вместо вывода карты, вставил картинку.

Теперь видно какие страны есть в индексе, а каких нет, и сколько городов в каждой стране. Меня всё устраивает, вот только мне не нужно в Индии так много городов. Удалю лишние (оставлю 2).

In [11]:
india = df.loc[(df['country'] == 'India') & 
              (df['city'] != 'Delhi') & 
              (df['city'] !='Mumbai')]

df = df.drop(india.index)

Подготовлю (индексы), чтобы проще было загрузить данные в БД, и сохраню эту таблицу, чтобы потом с нее по каждому городу скрапить данные в БД.

In [12]:
df.reset_index(drop=True, inplace=True)
df.index += 1
df.index.name = 'city_id'

df.to_csv("data/numbeo_main_table.csv")

# Таблица всех штатов США в формате ISO
Достану из википедии и сохраню таблицу кодов штатов США, тк это уже готовая таблица для будущей базы данных.

In [13]:
url_states = 'https://en.wikipedia.org/wiki/ISO_3166-2:US'

df_states = pd.read_html(url_states)[0]

df_states.columns = ['state_code', 'state_name', 'category']
df_states.set_index('state_code', inplace=True)

df_states.to_csv("data/states_codes.csv")

# Таблица всех стран и кодов

In [14]:
url_countries = 'https://en.wikipedia.org/wiki/ISO_3166-1'

df_countries = pd.read_html(url_countries)[1]
df_countries.rename(columns={'English short name (using title case)': 'country', 
                             'Alpha-3 code': 'country_code', 
                             'Independent': 'independent'}, inplace=True)
df_countries = df_countries.iloc[:, [0, 2, 5]]
df_countries.set_index('country_code', inplace=True)

for index, row in df_countries.iterrows():
    if index != 'VGB' and index != 'VIR':
        row['country'] = row['country'].strip('"')
        row['country'] = row['country'].split(' (')[0]
        row['country'] = row['country'].split('[')[0]
        row['country'] = row['country'].split(',')[0]
        
df_countries.to_csv("data/countries_codes.csv")